In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Put input_docs_sample.zip in your Google Drive

!rm -rf input_docs
!cp /content/drive/My\ Drive/input_docs.zip .
!unzip input_docs.zip > /dev/null
!ls input_docs/ | wc -l

# for the real collection change above input_docs_sample.zip to input_docs.zip
# for the sample collection of 5 docs, the process is fast
# for the real collection, the process takes about 6 min (start to finish, the whole notebook) 

19026


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [0]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs")

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os

#(did,text)
input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))
#normal_raw_data = input2.filter(lambda x: 'STANDARD OIL SRD TO FORM FINANCIAL UNIT' in x[1])
#normal_count = normal_raw_data.count()
#normal_count
print(input2.take(2))



[(12959, "<H2> 3-APR-1987 11:55:12.42</H2>\r\n<H2>POLYSAR IN JOINT VENTURE ON FINLAND LATEX PLANT</H2>\r\nPolysar Ltd, wholly owned by\nCanada Development Corp, said it agreed to form a joint\nventure with Raision Tehtaat, of Finland, to build a synthetic\nrubber latex plant in southern Finland.\n    Project cost and plant capacity were not disclosed.\n    The joint venture, to be 51 pct owned by Raision Tehtaat\nand 49 pct by Polysar, will build a plant at Anjalankoski, east\nof Helsinki, to produce carboxylated styrene-butadiene latex.\n    The plant's production will be used by Finland's paper\nindustry for making coated paper and paper board products.\n \n "), (9696, "<H2>25-MAR-1987 23:10:06.55</H2>\r\n<H2>CHINA, PORTUGAL INITIAL MACAO PACT</H2>\r\nChina and Portugal today initialled a\njoint declaration under which the 400-year-old colony of Macao\nwill be handed over to Peking on December 20, 1999, the\nofficial New China News Agency reported.\n    Portuguese Prime Minister Anib

In [0]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
porter = PorterStemmer()
lemma = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
# for a given doc return a list of tuples of the form (w, (docid, freq, freq/maxfreq))
def dw(docid, htmltext):
  cleantext = BeautifulSoup(htmltext).get_text()
  clean_text= re.findall('[a-zA-Z]+', cleantext)
  #clean_text = [item for item in clean_text if not item.isdigit()]
  words = [word.lower() for word in clean_text]
  wordsFiltered = []
  for w in words:
      if w not in stopWords:
          wordsFiltered.append(w)

  wordsFiltered=[lemma.lemmatize(word) for word in wordsFiltered]  

  unique_words = []
  for word in wordsFiltered:
      if word not in unique_words:
          unique_words += [word]
    

  word_frequencies = []
  for word in unique_words:
      word_frequencies.append(wordsFiltered.count(word))


  final_list=[]
  for i in range(len(unique_words)):
    final_list = final_list + [(unique_words[i],(docid,word_frequencies[i],(word_frequencies[i]/max(word_frequencies))))]
  return final_list
  #TODO
  #returning just a dummy result
  #return [(_,(_,_,_))]
  
word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0],x[1]))
print(word_docid_freq_tf.take(2))

  


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[('apr', (12959, 1, 0.2)), ('polysar', (12959, 3, 0.6))]


Expected output (all expected results are on the small sample):

<pre>
[('feb', (1, 1, 0.07142857142857142)), ('bahia', (1, 5, 0.35714285714285715))]
</pre>

In [0]:
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])

#TODO
word_postinglist_freq_tf=word_docid_freq_tf.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda p,q: p+q)
# creating a dummy RDD
# word_postinglist_freq_tf = sc.parallelize([ output ])
# word_postinglist_freq_tf = sc.parallelize([ ('test', [(1, 1, 0.5), (2, 1, 0.2)]) ])

In [0]:
print(word_postinglist_freq_tf.take(2))

[('apr', [(12959, 1, 0.2), (16400, 1, 0.125), (14357, 1, 0.125), (15275, 1, 0.2), (12027, 1, 0.3333333333333333), (15216, 1, 0.25), (13929, 1, 0.2), (13100, 1, 0.25), (15168, 1, 0.16666666666666666), (17042, 1, 0.16666666666666666), (16253, 1, 0.25), (13044, 1, 0.2), (15200, 1, 0.16666666666666666), (11847, 1, 0.1111111111111111), (15613, 1, 0.3333333333333333), (16098, 1, 0.3333333333333333), (16906, 1, 0.16666666666666666), (17301, 1, 0.1111111111111111), (16087, 1, 0.05263157894736842), (12844, 1, 0.3333333333333333), (17030, 1, 0.08333333333333333), (16056, 1, 0.3333333333333333), (16756, 1, 0.05263157894736842), (17380, 1, 0.14285714285714285), (14166, 1, 0.5), (13608, 2, 0.3333333333333333), (15528, 1, 0.5), (16205, 1, 0.125), (14906, 1, 0.1), (15423, 1, 0.25), (14331, 1, 0.1), (16146, 1, 0.1), (16778, 1, 0.16666666666666666), (12804, 1, 0.25), (12205, 1, 0.25), (13045, 1, 0.25), (17353, 1, 0.14285714285714285), (16231, 1, 0.2), (15507, 1, 0.25), (16758, 1, 0.25), (12202, 1, 0.33

Expected output

<pre>
[('feb', [(1, 1, 0.07142857142857142), (2, 1, 0.2), (5, 1, 0.16666666666666666), (3, 1, 0.3333333333333333), (4, 1, 0.07142857142857142)]), ('bahia', [(1, 5, 0.35714285714285715)])]
</pre>

In [0]:
# (word, [(did,freq,tfidf), ...])
# We easily obtain idf as 1/len(postinglist_tf)
# idf = 1/len(postinglist_tf)

list1= word_postinglist_freq_tf.collect()
list2=[]
for idx,i in enumerate(list1):
  list3=[]
  for j in i[1]:
    j=list(j)
    j[2]=(j[2]/len(i[1]))
    j=tuple(j)
    list3.append(j)
  list2.append((i[0],list3))
#TODO
word_postinglist_freq_tfidf = sc.parallelize(list2)
print(word_postinglist_freq_tfidf.take(8))




[('apr', [(12959, 1, 4.215851602023609e-05), (16400, 1, 2.6349072512647553e-05), (14357, 1, 2.6349072512647553e-05), (15275, 1, 4.215851602023609e-05), (12027, 1, 7.026419336706015e-05), (15216, 1, 5.269814502529511e-05), (13929, 1, 4.215851602023609e-05), (13100, 1, 5.269814502529511e-05), (15168, 1, 3.5132096683530073e-05), (17042, 1, 3.5132096683530073e-05), (16253, 1, 5.269814502529511e-05), (13044, 1, 4.215851602023609e-05), (15200, 1, 3.5132096683530073e-05), (11847, 1, 2.3421397789020048e-05), (15613, 1, 7.026419336706015e-05), (16098, 1, 7.026419336706015e-05), (16906, 1, 3.5132096683530073e-05), (17301, 1, 2.3421397789020048e-05), (16087, 1, 1.109434632111476e-05), (12844, 1, 7.026419336706015e-05), (17030, 1, 1.7566048341765037e-05), (16056, 1, 7.026419336706015e-05), (16756, 1, 1.109434632111476e-05), (17380, 1, 3.011322572874006e-05), (14166, 1, 0.00010539629005059021), (13608, 2, 7.026419336706015e-05), (15528, 1, 0.00010539629005059021), (16205, 1, 2.6349072512647553e-05)

Expected output

<pre>
[('feb', [(1, 1, 0.014285714285714285), (2, 1, 0.04), (5, 1, 0.03333333333333333), (3, 1, 0.06666666666666667), (4, 1, 0.014285714285714285)])]
</pre>

In [0]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 

#TODO

temp4=word_postinglist_freq_tfidf.map(lambda x: x[1][:]).flatMap(lambda x: [(i[0],(i[1],i[2]**2)) for i in x[:]]).collect()
did_freq_tfidfsq_rdd = sc.parallelize(temp4)
# RDD of (did,(freq,tfidf)) tuples
# creating a dummy RDD
print(did_freq_tfidfsq_rdd.take(5))
temp5= did_freq_tfidfsq_rdd.reduceByKey(lambda x,y:(max(x[0],y[0]),x[1] + y[1])).collect()



# Produce (did,(maxf,magnitudesq))
# creating a dummy RDD
doc_maxf_mag = sc.parallelize(temp5)

print(doc_maxf_mag.take(5))

[(12959, (1, 1.7773404730285028e-09)), (16400, (1, 6.942736222767588e-10)), (14357, (1, 6.942736222767588e-10)), (15275, (1, 1.7773404730285028e-09)), (12027, (1, 4.9370568695236195e-09))]
[(16400, (8, 0.0034821916809178732)), (15216, (4, 0.04384375136310996)), (13100, (4, 1.0009134527283605)), (15168, (6, 0.007492377558908097)), (13044, (5, 0.13502771593329124))]


Excpected result

<pre>
[(1, (1, 0.0002040816326530612)), (2, (1, 0.0016))]
[(2, (5, 3.894100000000001)), (4, (14, 2.94553429705215))]
</pre>

In [0]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [0]:

!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [0]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/My\ Drive/inv_idx.txt
!wc -l /content/drive/My\ Drive/inv_idx.txt

total 36876
-rw-r--r-- 1 root root   538757 Apr  3 04:58 part-00003
-rw-r--r-- 1 root root   641774 Apr  3 04:58 part-00001
-rw-r--r-- 1 root root 16747067 Apr  3 04:58 part-00002
-rw-r--r-- 1 root root 19828641 Apr  3 04:58 part-00000
-rw-r--r-- 1 root root        0 Apr  3 04:58 _SUCCESS
('jkpt', [(5406, 1, 0.2)])
('companmy', [(9246, 1, 0.125)])
('computrac', [(16574, 3, 0.75)])
('reshaping', [(15478, 1, 0.1111111111111111), (17862, 1, 0.030303030303030304), (2924, 1, 0.030303030303030304)])
('telemecanique', [(19068, 3, 0.75)])
('rac', [(15899, 1, 0.05555555555555555), (15907, 1, 0.05555555555555555), (16022, 1, 0.06666666666666667)])
('fivefold', [(21398, 1, 0.125), (19918, 1, 0.038461538461538464)])
('stipulate', [(7643, 1, 0.07692307692307693)])
('pacesetting', [(13759, 1, 0.125), (14358, 1, 0.125)])
('kahle', [(2445, 1, 0.2)])
9216 inv_idx/part-00000
10240 inv_idx/part-00001
19456 /content/drive/My Drive/inv_idx.txt


In [0]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/My\ Drive/doc_mag.txt
!wc -l /content/drive/My\ Drive/doc_mag.txt

total 644
-rw-r--r-- 1 root root 139755 Apr  3 04:58 part-00000
-rw-r--r-- 1 root root 174647 Apr  3 04:58 part-00002
-rw-r--r-- 1 root root 174656 Apr  3 04:58 part-00001
-rw-r--r-- 1 root root 159952 Apr  3 04:58 part-00003
-rw-r--r-- 1 root root      0 Apr  3 04:58 _SUCCESS
(16400, (8, 0.0034821916809178732))
(15216, (4, 0.04384375136310996))
(13100, (4, 1.0009134527283605))
(15168, (6, 0.007492377558908097))
(13044, (5, 0.13502771593329124))
(15200, (6, 0.0658873516237678))
(12844, (3, 0.003977463691551604))
(16056, (3, 0.3180168811959522))
(16756, (19, 0.008604643672558043))
(17380, (7, 0.02325186713681282))
4096 doc_mag/part-00000
5120 doc_mag/part-00001
9216 /content/drive/My Drive/doc_mag.txt


**Interface for Query Search By Cosine Similarity below**

In [0]:
ls=[]
total_vocab=[]
for i in list2:
  total_vocab.append(i[0])
  for j in i[1]:
    ls.append(((j[0],i[0]),j[2]))
ls_req=dict(ls)


temp_cos=word_docid_freq_tf.collect()
ls3=[]
for i in temp_cos:
  ls3.append((i[0],i[1][2]))

In [0]:
import numpy as np
import math
N=len(input.collect())
# N=21578
M=len(total_vocab)
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [0]:
def cleaning(cleantext):
  clean_text= re.findall('[a-zA-Z]+', cleantext)
  #clean_text = [item for item in clean_text if not item.isdigit()]
  words = [word.lower() for word in clean_text]
  wordsFiltered = []
  for w in words:
      if w not in stopWords:
          wordsFiltered.append(w)
  wordsFiltered=[lemma.lemmatize(word) for word in wordsFiltered]
  return wordsFiltered

In [0]:

def gen_vector(tokens):

    Q = np.zeros(M)
    # Q = dia_matrix(M, dtype=np.int8).toarray()
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = dict(ls3)[token]
        idf = (math.log((N+1)/(df+1)))
        ind=total_vocab.index(token)
        Q[ind] = tf*idf
    return Q


In [0]:
D = np.zeros((N, M))
for i in ls_req:
  ind = total_vocab.index(i[1])
  D[i[0]-1][ind-1] = ls_req[i]


In [0]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    tokens = cleaning(query)
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(d, query_vector))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    print("Docids in the decreasing order of relevance ")
    print("")
    print(out+1)

Q = cosine_similarity(5, "BankAmerica Corp is not under\npressure to act quickly on its proposed equity offering and\nwould do well to delay it because of the stock\'s recent poor\nperformance, banking analysts said")

Cosine Similarity

Query: BankAmerica Corp is not under
pressure to act quickly on its proposed equity offering and
would do well to delay it because of the stock's recent poor
performance, banking analysts said

['bankamerica', 'corp', 'pressure', 'act', 'quickly', 'proposed', 'equity', 'offering', 'would', 'well', 'delay', 'stock', 'recent', 'poor', 'performance', 'banking', 'analyst', 'said']

Docids in the decreasing order of relevance 

[4 3 1 2 5]
